In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re

In [2]:
main_url='https://www.imdb.com/feature/genre/?ref_=nv_ch_gr'

In [3]:
source=requests.get(main_url).text
soup=BeautifulSoup(source,'lxml')

In [4]:
for x in  soup.find_all('div',class_='article'):
    print(x.find('span',class_='oneline').parent.parent.prettify())

<div class="ab_ninja">
 <span class="widget_header">
  <span class="oneline">
   <h3>
    Popular TV Show and Movie Genres
   </h3>
  </span>
 </span>
 <div class="widget_content no_inline_blurb">
  <div class="widget_nested">
   <div class="ninja_image_pack">
    <div class="ninja_left">
     <div class="ninja_image first_image" style="width:31.952%;height:auto;">
      <div style="width:100%;height:auto;margin:0 auto;">
       <div class="widget_image">
        <div class="image">
         <a href="https://www.imdb.com/search/title?genres=comedy&amp;explore=title_type,genres">
          <img alt="Comedy" class="pri_image" src="https://m.media-amazon.com/images/G/01/IMDb/genres/Comedy._CB1513316167_SX233_CR0,0,233,131_AL_.jpg" title="Comedy"/>
         </a>
        </div>
       </div>
      </div>
     </div>
     <div class="ninja_image ninja_image_relative_padding widget_padding" style="width:2%;">
     </div>
     <div class="ninja_image" style="width:31.952%;height:auto;">
      

AttributeError: 'NoneType' object has no attribute 'parent'

In [ ]:
genre=[]
link_page=[]
for x in soup.find_all('span',class_='oneline'):
    head=x.h3.text
    if head==' Popular Movies by Genre':
        par=x.parent.parent
        for genres in par.find_all('div',class_='table-cell'):
            genre.append(genres.a.text.strip())
            link_page.append('http://imdb.com'+ genres.a['href'])

In [ ]:
df= pd.DataFrame({'genre':genre,'link_page':link_page})

In [ ]:
df['genre']=df['genre'].apply(lambda x : '-'.join(x.split(' ')))

In [ ]:
df

<b> removing 'super hero' genre as it needs to be handled alone </b>

In [ ]:
df_super=df[df['genre']=='Superhero']

In [ ]:
df_super

In [ ]:
indexNames = df[df['genre']=='Superhero'].index
# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)
df=df.drop_duplicates()
df=df.reset_index(drop=True)

In [ ]:
df

### Get and parse a page

In [ ]:
def getpage(url):
    source=requests.get(url).text
    soup=BeautifulSoup(source,'lxml')
    return(soup)

### To get all movie title codes in a single page

In [ ]:
def get_titlecodes_from_page(url):
    lst1=[] 
    soup=getpage(url)
    for x in soup.find_all('div',class_='lister-item mode-advanced'):
        lst1.append(x.a['href'].split('/')[-2])
    return(lst1)   

### Getting for a single genre

In [ ]:
df['link_page'][22]

In [ ]:
url = df['link_page'][22]
soup = getpage(url)
lst2 = []
lst2 = get_titlecodes_from_page(url)
pages2 = soup.find('div', class_='desc').span.text.split(' ')
total2 = int(''.join(pages2[-2].split(',')))
looptimes2 = int(total2/50)
for pg in range(looptimes2):
    nexturl2 = 'https://www.imdb.com/search/title/?title_type=feature&genres='+df['genre'][22]+'&start=' + \
        str((pg+1)*50+1)+'&explore=genres&ref_=adv_nxt'
    lst2.extend(get_titlecodes_from_page(nexturl2))

In [ ]:
len(lst2)

### getting codes for all genre

In [ ]:
df['codes']=0
df['titles_cont']=0

In [ ]:
all_codes=[]
for num in range(df.shape[0]):
    url = df['link_page'][num]
    soup = getpage(url)
    lst = []
    lst = get_titlecodes_from_page(url)
    pages = soup.find('div', class_='desc').span.text.split(' ')
    total = int(''.join(pages[-2].split(',')))
    looptimes = int(total/50)
    for pg in range(looptimes):
        nexturl = 'https://www.imdb.com/search/title/?title_type=feature&genres=' + \
            df['genre'][num]+'&start=' + \
            str((pg+1)*50+1)+'&explore=genres&ref_=adv_nxt'
        lst.extend(get_titlecodes_from_page(nexturl))
        all_codes.extend(lst)
        df['codes'][num] = lst
        df['titles_cont'][num] = len(lst)

In [ ]:
df[['genre','titles_cont']]

### saving csv file

In [ ]:
df.to_csv('IMDb_movies_codes1.csv',index=False)

In [ ]:
df1=pd.DataFrame({'codes':all_codes})
df1.to_csv('IMDB_movie_all_codes.csv',index=False)